<a href="https://colab.research.google.com/github/sfnesbit/CSC466-Team13/blob/main/CSC_466_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC 466 Team 13 Food Image Classification
Ty Farris | Marine Cossoul | Sean Nesbit

## Mount drive
*You may need to visit CSC466 folder, and 'Add Shortcut to Drive' or 'Move To > My Drive"*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/CSC466/data
!unzip drive/MyDrive/CSC466/data/food.zip -d drive/MyDrive/CSC466/dataset

## Load data from shared drive and unzip the data into a pandas dataframe

In [ ]:
import pandas as pd
import os
